In [ ]:
from pathlib import Path
import pandas as pd
import os
import scipy
import numpy as np


In [ ]:
# Define the folder path where the CSV file is located
folder_path = r"S:\microscopy\Calcium imaging\24.02.23-29 srx-9_FLP\analysis_paper\bout_analysis"
# Define the base name for the file
file_name = 'intensity_and_speed_immobiliy_df_'
# Define the strain name (e.g., "3451") to create a specific file name
strain_name = '3451'
# Construct the full file name by combining the base file name with the strain name
strain_file_name =  file_name + strain_name + ".csv"
# Read the CSV file into a pandas DataFrame using the full file path
data_df = pd.read_csv(os.path.join(folder_path, strain_file_name))
# Display the contents of the DataFrame
data_df

In [ ]:
# Group the data by 'replicate' and 'n_worm' to analyze each worm separately
grouped_worms = worm_data_df.groupby(['replicate','n_worm'])
# Sort each group by 'time' to ensure temporal order before further processing
sorted_groups = grouped_worms.apply(lambda x: x.sort_values(by='time')).reset_index(drop=True)
# Re-group the sorted data for further transformations
regrouped_sorted = sorted_groups.groupby(['replicate', 'n_worm'])

# Interpolate missing values in 'normalised_intensity' and 'speed' using linear interpolation
worm_data_df['normalised_intensity'] = regrouped_sorted['normalised_intensity'].transform(lambda x: x.interpolate(method='linear', limit_direction='both'))
worm_data_df['speed'] = regrouped_sorted['speed'].transform(lambda x: x.interpolate(method='linear', limit_direction='both'))

# Re-group the data again after interpolation
grouped_worms = worm_data_df.groupby(['replicate','n_worm'])

# Apply Savitzky-Golay filtering for smoothing:
# - 'smooth_intensity_hard': Intensity smoothed with a larger window (120 frames) for stronger smoothing
# - 'smooth_intensity_soft': Intensity smoothed with a smaller window (20 frames) for less aggressive smoothing
# - 'smooth_speed': Speed smoothed with a window of 20 frames
worm_data_df['smooth_intensity_hard'] = grouped_worms['normalised_intensity'].transform(lambda x: scipy.signal.savgol_filter(x, 120, 0, mode='mirror'))
worm_data_df['smooth_intensity_soft'] = grouped_worms['normalised_intensity'].transform(lambda x: scipy.signal.savgol_filter(x, 20, 0, mode='mirror'))
worm_data_df['smooth_speed'] = grouped_worms['speed'].transform(lambda x: scipy.signal.savgol_filter(x, 20, 0, mode='mirror'))

In [ ]:
# Detect peaks in the 'smooth_intensity_hard' signal for each worm in the dataset
peaks_list = grouped_worms.apply(
    lambda group: scipy.signal.find_peaks(
        group['smooth_intensity_hard'],  # Input signal to analyze
        prominence=0.02,  # Minimum prominence (height difference from surrounding points) to detect peaks
        width=0  # Minimum peak width (0 means no restriction)
    )
)

# Display the detected peaks for each worm group
peaks_list

In [ ]:
# Assigns a new unique 'n_worm_new' ID for each worm within the same time point
worm_data_df['n_worm_new'] = worm_data_df.groupby('time').cumcount() + 1

# Create matrices (pivot tables) for different parameters over time, with worms as columns
# The values are smoothed intensity, speed, and binary sleep state

# Pivot table for smoothed intensity ('smooth_intensity_soft')
intensity_matrix = worm_data_df.pivot(index='time', columns='n_worm_new', values='smooth_intensity_soft')
# Pivot table for smoothed speed ('smooth_speed')
speed_matrix = worm_data_df.pivot(index='time', columns='n_worm_new', values='smooth_speed')
# Pivot table for sleep state ('binary_table')
sleep_matrix = worm_data_df.pivot(index='time', columns='n_worm_new', values='binary_table')

In [ ]:
# Initialize dictionaries and lists for storing adjusted peak data
adjusted_peak_dict = {}
adjusted_peak_dict = {}
adjusted_peak_dict_list = []

# Get unique worm IDs
worms_count = worm_data_df['n_worm_new'].unique().tolist()

# Loop over each worm and its corresponding detected peaks
for n_worm, peak_sublist in enumerate(peaks_list.to_list(), start=1):
    adjusted_peak_list = []

    # Iterate over detected peaks and their widths
    for peak, width in zip(peak_sublist[0], peak_sublist[1]['widths']) :
        left_limit = int(peak - width/2)-25
        right_limit = int(peak + width/2)+25

        # Ensure limits stay within bounds
        if left_limit < 0:
            left_limit = 0
        if right_limit > len(intensity_matrix):
            right_limit = len(intensity_matrix)
        index_range = slice(left_limit, right_limit)
        selected_sublist = intensity_matrix.iloc[:, (n_worm-1)][index_range]

        # Find the index of the maximum intensity within the selected range (adjusted peak)
        adjusted_peak_list.append(selected_sublist.idxmax())
        
    # Store adjusted peak information in a dictionary
    adjusted_peak_dict = {
        'n_worm_new': worms_count[n_worm - 1],  # Assign the corresponding worm ID
        'peaks': list(sorted(set(adjusted_peak_list))),  # Adjusted peak locations
        'peaks_old': list(sorted(set(peak_sublist[0])))  # Original peak locations
    }

    # Append the dictionary to the list
    adjusted_peak_dict_list.append(adjusted_peak_dict)

# Convert the list of dictionaries into a DataFrame
new_peaks_df = pd.DataFrame(adjusted_peak_dict_list)  
new_peaks_df


In [ ]:
# Flatten the lists in 'peaks' and 'peaks_old' columns
all_peaks = new_peaks_df['peaks'].explode().tolist() 

# Calculate the total count of all peaks
total_peaks_count = len(all_peaks)

# Display the total count
print("Total Peaks Count:", total_peaks_count)

In [ ]:
new_peaks_df['num_peaks'] = new_peaks_df['peaks'].apply(len)

# Specify the filename for saving the DataFrame to a CSV file
peaks_raw = f'{strain_name}_peaks_x_position.csv'
# Create the full path for saving the CSV file
peaks_position_saving_path = os.path.join(folder_path, peaks_raw)
# Save the DataFrame to a CSV file at the specified path
new_peaks_df.to_csv(peaks_position_saving_path, index=False)


In [ ]:
# Remove the 'peaks' and 'peaks_old' columns from new_peaks_df to create peak_data_df
peak_data_df = new_peaks_df.drop(['peaks', 'peaks_old'], axis=1)

# Get the 'replicate' value for each worm by taking the first occurrence within each worm group
replicate_per_worm = worm_data_df.groupby('n_worm_new')['replicate'].first()
# Map the replicate values to the corresponding worms in peak_data_df
peak_data_df['replicate'] = peak_data_df['n_worm_new'].map(replicate_per_worm)
# Calculate the peak frequency per minute (assuming 5 minutes of observation time)
peak_data_df['peaks_frequency'] = peak_data_df['num_peaks'] / 5

# Assign strain name (commented out, can be enabled if needed)
# peak_data_df['strain'] = strain_name

# Specify the filename for saving the DataFrame to a CSV file
peaks_to_save = f'{strain_name}_peak_data.csv'

# Create the full path for saving the CSV file
peaks_saving_path = os.path.join(folder_path, peaks_to_save)

# Save the DataFrame to a CSV file at the specified path
peak_data_df.to_csv(peaks_saving_path, index=False)

In [ ]:

# Define time window before and after each detected peak
time_before_peak = 150
time_after_peak = 350
total_time = time_before_peak + time_after_peak

# Initialize lists to store aligned matrices for peak intensity, speed, and sleep probability
aligned_peaks_matrix_of_worms = []
aligned_speed_matrix_of_worms = []
aligned_sleep_matrix_of_worms = []

# Retrieve the list of worm identifiers from 'new_peaks_df'
worms = new_peaks_df['n_worm_new'].values.tolist()

# Iterate over each worm in the dataset
for worm in worms:
    # Create empty matrices to store aligned peaks and speed data
    aligned_peaks_matrix = np.empty((0, (total_time+1)))
    aligned_speed_matrix = np.empty((0, (total_time+1)))
    aligned_sleep_matrix = np.empty((0, (total_time+1)))
    
    # Filter peak data for the current worm
    filtered_peaks = new_peaks_df.loc[new_peaks_df['n_worm_new'] == worm, 'peaks']
    worm_df = worm_data_df[worm_data_df['n_worm_new'] == worm]  # Filter DataFrame for worm
    
    # Convert filtered peak data into a list
    peaks_list = filtered_peaks.values.tolist()[0]
    if len(peaks_list) == 0: # Skip worms that have no detected peaks
        continue
        
    # Iterate over each detected peak for the current worm
    for peak in peaks_list:
        # Define the start and end indices for slicing data around the peak
        x_start, x_end = peak - time_before_peak, peak + time_after_peak
        
        # Extract data for intensity, speed, and sleep probability from their respective matrices
        original_bout = np.array([intensity_matrix.loc[x_start:x_end, worm].values])
        original_speed = np.array([speed_matrix.loc[x_start:x_end, worm].values])
        original_sleep = np.array([sleep_matrix.loc[x_start:x_end, worm].values])
        
        # Handle boundary cases where indices fall outside the available data range
        if x_start < 0:
             # Pad with NaNs if the extracted range starts before the first time point
            bout = (np.append(np.repeat(np.nan, abs(x_start)), original_bout)).reshape(1,total_time+1)
            speed = (np.append(np.repeat(np.nan, abs(x_start)), original_speed)).reshape(1,total_time+1)
            sleep = (np.append(np.repeat(np.nan, abs(x_start)), original_sleep)).reshape(1,total_time+1)
            aligned_peaks_matrix = np.append(aligned_peaks_matrix, bout, axis = 0)
            aligned_speed_matrix = np.append(aligned_speed_matrix, speed, axis = 0)
            aligned_sleep_matrix =  np.append(aligned_sleep_matrix, sleep, axis = 0)
        
        elif x_end > len(intensity_matrix)-1:
            # Pad with NaNs if the extracted range exceeds the last available time point
            bout = (np.append(original_bout, np.repeat(np.nan, (x_end - len(worm_df)+1)))).reshape(1,total_time+1)
            speed = (np.append(original_speed, np.repeat(np.nan, (x_end - len(worm_df)+1)))).reshape(1,total_time+1)
            sleep = (np.append(original_sleep, np.repeat(np.nan, (x_end - len(worm_df)+1)))).reshape(1,total_time+1)
            aligned_peaks_matrix = np.append(aligned_peaks_matrix, bout, axis = 0)
            aligned_speed_matrix = np.append(aligned_speed_matrix, speed, axis = 0)
            aligned_sleep_matrix = np.append(aligned_sleep_matrix, sleep, axis = 0)
        
        else:
            # Append the extracted data directly if within the available range
            aligned_peaks_matrix = np.append(aligned_peaks_matrix, original_bout, axis = 0)
            aligned_speed_matrix = np.append(aligned_speed_matrix, original_speed, axis = 0)
            aligned_sleep_matrix = np.append(aligned_sleep_matrix, original_sleep, axis = 0)
            
    # Compute the mean-aligned matrices across all detected peaks for this worm
    aligned_peaks_matrix_of_worms.append(np.nanmean(aligned_peaks_matrix, axis = 0))
    aligned_speed_matrix_of_worms.append(np.nanmean(aligned_speed_matrix, axis = 0))
    aligned_sleep_matrix_of_worms.append(np.nanmean(aligned_sleep_matrix, axis=0))
                
#np.savetxt(os.path.join(folder_path, strain_name+'_aligned_peaks_worm.csv'), aligned_peaks_matrix_of_worms, delimiter=",")

#np.savetxt(os.path.join(folder_path, strain_name+'_aligned_speed_worm.csv'), aligned_speed_matrix_of_worms, delimiter=",")

aligned_sleep_matrix_of_worms_to_save = [1-x for x in aligned_sleep_matrix_of_worms]
#np.savetxt(os.path.join(folder_path, strain_name+'_aligned_sleep_worm.csv'), aligned_sleep_matrix_of_worms_to_save, delimiter=",")     
        



In [ ]:
# Create a list to hold each row of data
data_rows = []

# Iterate through each worm's data
for worm_peak, worm_speed, worm_sleep, i in zip(aligned_peaks_matrix_of_worms, aligned_speed_matrix_of_worms, aligned_sleep_matrix_of_worms, worms_count):
    
    # Iterate through time points and store data
    for j, (intensity, speed, sleep) in enumerate(zip(worm_peak, worm_speed, worm_sleep)):
        data_rows.append({'n_worm_new': i, 'time': j, 'intensity': intensity, 'speed':speed, 'sleep':(1-sleep), 'strain': strain_name})

# Create DataFrame
worm_bout_df = pd.DataFrame(data_rows)

# Specify the filename for saving the DataFrame to a CSV file
worm_bout_to_save = f'{strain_name}_each_worm_peak_{time_after_peak}_min.csv'

# Create the full path for saving the CSV file
worm_bout_to_save_path = os.path.join(folder_path, worm_bout_to_save)

# Save the DataFrame to a CSV file at the specified path
worm_bout_df.to_csv(worm_bout_to_save_path, index=False)

worm_bout_df


In [ ]:
import math

# Calculate the number of valid (non-NaN) entries for each time point across all worms
valid_counts = np.sum(~np.isnan(aligned_sleep_matrix_of_worms), axis=0) 

# Calculate the mean of the aligned peaks across all worms (ignoring NaNs)
aligned_peaks = np.nanmean(aligned_peaks_matrix_of_worms, axis = 0)
# Calculate the standard deviation of aligned peaks and compute the error using the standard error of the mean
error_peaks = np.nanstd(aligned_peaks_matrix_of_worms, axis=0)/np.sqrt(valid_counts)
# Calculate the mean of the aligned speed values across all worms (ignoring NaNs)
aligned_speed = np.nanmean(aligned_speed_matrix_of_worms, axis = 0)
# Calculate the standard deviation of aligned speed values and compute the error using the standard error of the mean
error_speed = np.nanstd(aligned_speed_matrix_of_worms, axis=0)/np.sqrt(valid_counts)

# Calculate the sleep probability by computing the average across all worms (ignoring NaNs) and subtracting from 1
aligned_sleep = 1 - np.nansum(aligned_sleep_matrix_of_worms, axis=0)/ valid_counts

In [ ]:
# Combine the data into a dictionary
data = {
    'Mean Vector Speed': aligned_speed,  # Mean speed for each time point
    'Error Speed': error_speed,  # Standard error for the speed values
    'Mean Vector Intensity': aligned_peaks,  # Mean intensity for each time point
    'Error Intensity': error_peaks,  # Standard error for the intensity values
    'Sleep probability': aligned_sleep  # Sleep probability (1 - wakefulness)
}

# Create a pandas DataFrame from the data dictionary
saving_df = pd.DataFrame(data)

# Specify the filename for saving the DataFrame to a CSV file
file_to_save = f'{strain_name}_peak_alignment_{time_after_peak}min.csv'

# Create the full path for saving the CSV file
saving_path = os.path.join(folder_path, file_to_save)

# Save the DataFrame to a CSV file at the specified path
saving_df.to_csv(saving_path)